In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
#importing libraries
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("../input/car-price-prediction-challenge/car_price_prediction.csv")

In [4]:
df.head()

In [5]:
df.shape

In [6]:
df.info()

Dataset have no null values and most of the variables are object types 

In [7]:
df.describe()

this distribution above shows that numerical variables have no such outliers 

**Data cleaning and EDA**

In [8]:
df=df.drop_duplicates()

In [9]:
df.shape

some duplicate values are present in the dataset

In [10]:
df["Doors"].value_counts()

values in the Doors column was not properly given .so need to correct  into proper format  

In [11]:
df["door1"],df["door2"]=df['Doors'].str.split('-').str
df['door1']=df['door1'].str.lstrip('0')

In [12]:
d = {'Mar':3, 'May':5 }
df.door2 = df.door2.map(d)

In [13]:
df["door2"] = df["door2"].fillna(0.0).astype(int)

In [14]:
df["Doors"]=df["door1"].str.cat(df[["door2"]].astype(str), sep="-")

In [15]:
df.drop(["door1","door2"],axis=1,inplace=True)
df.drop(["ID"],axis=1,inplace=True)
df.drop(['Levy'],axis=1,inplace=True)

Levy column in this dataset given some importing and exporting values .it seems to be imporper for future dataanalysis.so dropping irrelevent columns ID and Levy 

In [16]:
d = []
for i in df['Mileage']:
  d.append(int(i.split()[0]))
df['Mileage'] = d

columns Mileage,Engine volume are numerical variable but its datatype given as object .for future analysis its dtype should be in proper format 

In [17]:
a = []
for i in df['Engine volume']:
  i = i.split()[0]
  a.append(float(i))
df['Engine volume'] = a

In [18]:

columns_with_na=[x for x in df.columns if df[x].isnull().sum()>1]

clarified that this dataset has no null values

**Numerical Data Analysis**

In [19]:
numerical_variable= df.select_dtypes(include=['float64', 'int64'])
#print("number of numerical variables:",len(numerical_variable))
numerical_variable.head()

In [20]:
for x in numerical_variable:
    print(x,df[x].unique())

In [21]:
df.groupby('Prod. year')['Price'].mean().plot()
plt.xlabel('Production Year')
plt.ylabel('mean Car Price')
plt.title('car price vs Production Year')

showing the distribution of car price according to production year .Price value very high in year between 1940-1950 and 1990-2000

**Discrete Variable Analysis**

In [22]:
disceate_variables=[x for x in numerical_variable if len(df[x].unique())<25]
print("discrete variable count: {}".format(len(disceate_variables)))
print(disceate_variables)

In [23]:
df[disceate_variables].head()

In [24]:
for x in disceate_variables:
    data=df.copy()
    data.groupby(x)["Price"].mean().plot.bar()
    plt.xlabel(x)
    plt.ylabel("Price")
    plt.title(x)
    plt.show()

**Countinous Variable Analysis**

In [25]:
continuous_variable=[x for x in numerical_variable if x not in disceate_variables]
print("continuous_variable: {}".format(len(continuous_variable)))


In [26]:
df[continuous_variable].head()

In [27]:
for x in continuous_variable:
    data=df.copy()
    data[x].hist(bins=10)
    plt.xlabel(x)
    plt.ylabel("count")
    plt.title(x)
    plt.show()

contionous variables are not aobeying normal distribution .so transformed in to log normal distribution 

In [28]:

for x in continuous_variable:
    data=df.copy()
    if 0 in data[x].unique():
        pass
    else:
        data[x]=np.log(data[x])
        data["Price"]=np.log(data["Price"])
        plt.scatter(data[x],data['Price'])
        plt.xlabel(x)
        plt.ylabel("Price")
        plt.title(x)
        plt.show()

In [29]:
for x in continuous_variable:
    data=df.copy()
    if 0 in data[x].unique():
        pass
    else:
        data[x]=np.log(data[x])
        data.boxplot(column=x)
        plt.ylabel(x)
        plt.title(x)
        plt.show
        

**Distribution of Target Column**

In [30]:
#df["Price"]=np.log(df["Price"])
sns.distplot(df["Price"])

**Correlation between the Variables**

In [31]:
corrmat = df.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20,20))
#plot heat map
g=sns.heatmap(df[top_corr_features].corr(),annot=True,cmap="RdYlGn")

**Categorial Value Analysis**

In [32]:
categorical_variable=[x for x  in df.columns if df[x].dtype=="O"]


In [33]:
df[categorical_variable].head()

In [34]:
for x in categorical_variable:
    print(x,len(df[x].unique()))

In [35]:
for x in categorical_variable:
    if (len(df[x].unique())>20):
        pass
    else:
   
        data=df.copy()
        data.groupby(x)['Price'].mean().plot.bar()
        plt.xlabel(x)
        plt.ylabel("Price")
        plt.title(x)
        plt.show()

In [36]:
df.drop(["Manufacturer","Model"],axis=1,inplace=True)

Manufacturer,Model are irrelevant for data analysis

In [37]:
df.info()

In [38]:
df.head()

**One hot Encoding for Categorical Variable**

In [39]:
df=pd.get_dummies(df,drop_first=True)


**Independent and dependent variable split **

In [40]:
X=df.loc[:,df.columns != 'Price']
y=pd.DataFrame(df['Price'])

In [41]:
y.head()

**feature scaling**

In [42]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()


In [43]:
x_list=['Prod. year','Engine volume','Mileage','Cylinders','Airbags']
X[x_list]=scaler.fit_transform(X[x_list])


In [44]:
X.head()

In [45]:
y=scaler.fit_transform(y)
y

**Feature Importance**

In [46]:


from sklearn.ensemble import ExtraTreesRegressor

model = ExtraTreesRegressor()
model.fit(X,y)

ExtraTreesRegressor improve the predictive accuracy and control over-fitting.


In [47]:
print(model.feature_importances_)

In [48]:
#plot graph of feature importances for better visualization
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

**Train and Test Split**

In [49]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

**Model1:Linear Regression**

In [50]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

In [51]:
lin_regressor=LinearRegression()
mse=cross_val_score(lin_regressor,X_train,y_train,scoring='neg_mean_squared_error',cv=5)
mean_mse=np.mean(mse)
print(mean_mse)

**Model2:Ridge Regression**

In [52]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

In [53]:
ridge=Ridge()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40]}
ridge_regressor=GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error',cv=5)
ridge_regressor.fit(X,y)

In [54]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

**Model3:Lasso**

In [55]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

In [56]:
lasso=Lasso()
parameters={'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40]}
lasso_regressor=GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error',cv=5)

lasso_regressor.fit(X_train,y_train)
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

**Model Evaluation**

In [57]:
prediction=ridge_regressor.predict(X_test)

For prediction ,took lasso regression it is useful for prevention of overfitting  

In [58]:
sns.distplot(y_test-prediction)

In [59]:
plt.scatter(y_test,prediction)

**Evaluation Metrics**

In [60]:
from sklearn import metrics
print('MAE:', metrics.mean_absolute_error(y_test, prediction))
print('MSE:', metrics.mean_squared_error(y_test, prediction))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, prediction)))